<a href="https://colab.research.google.com/github/samuelch3/stuco/blob/main/build_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# usage
1.   Enter the data in cell 1
2.   press "run all"
3.   when google asks if you want to grant access, click "yes" to everything
4.   watch the new stuco list populate
5.   on the new stuco list, clean up the FCE opt-in column to TRUE/FALSE instead of "Yes/No"
5.   make the formatting look nice on the new stuco list

# Update log:
* code: 2025-03-17
* Documentation: 2025-01-22

## 2025-03-17
* code: changed so that new courses are assigned 129xx numbers (CEE) as placeholder, so that stuco actually has to look at the numbers before hitting send
* working on Python version 3.11.11

## 2025-01-21
* code: updated to use values for F25 semester
* working on Python version 3.11.11

In [ ]:
import sys

print(sys.version)

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


# modify this cell

In [ ]:
# urls

# application form responses, either the id (gibberish string) or the url works
application_form = "https://docs.google.com/spreadsheets/d/1S9diX0iRztClihnpjP3-fmWAEsn6Kg8fpKqS7eNe_Bo"

# instructor list
# - create a new one (set to None) OR
# - input a link as above to write the instructor list to an existing sheet
previous_stuco_list_url = 'https://docs.google.com/spreadsheets/d/1il1kx7-Y88_7KPZJ3UCMF6OW6_0NhUf4AKxRPzIT3Gw'
new_stuco_list_url = "https://docs.google.com/spreadsheets/d/1rijdL1vZbRYvnOla8lpRQPRnvlqtkBHwYvZhn16v4uk"
new_semester_folder_url = "https://drive.google.com/drive/folders/1uN3gYa4TDyTLBdDNyaig38-hs_MRMLHJ"
folder_id = "1uN3gYa4TDyTLBdDNyaig38-hs_MRMLHJ"

semester = "F25"

# you need not touch the rest
(unless you change the application form)

log in to google drive to access the sheet

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

old_course_profiles = gc.open_by_url(previous_stuco_list_url).get_worksheet(1).get_all_records()

functions to process each application

In [ ]:
def gen_course_num(app, i):
  # listed returning course
  if app["What course are you making the request for?"] not in ("Not Listed", ""):
    return app['What course are you making the request for?'][:6], i
  # listed returning course
  elif app["What course are you making the request for?"] == "Not Listed":
    return app["If your course was not listed above, please fill it in below."][:6], i
  # new course: assign arbitrary 98-5xx number, to be replaced later
  else:
    return "12-" + str(i), i+1

def course_profile(app, course_num):
  profile = dict()
  new = app["Are you applying for a returning StuCo course or new StuCo course?"].startswith("New")
  change_profile = app["I verify that I have reviewed my course profile details on SIO/S3, and no changes are necessary at this time."].startswith("I'd like to make changes")
  if change_profile or new:
    course_profile_from_app(profile, app)
    if new: # get titles from current application if its a new course
      profile["Short Title"] = "STUCO: " + app["Short Course Name (for SIO)"].upper()
      profile["Long Title"] = app["Full Course Name"]
      return profile
  # will need to get short and long title from old list
  old_course_profile = None
  for course in old_course_profiles:
    if course["Class Number"] == course_num: old_course_profile = course; break
  if old_course_profile != None:
    profile["Short Title"] = old_course_profile["Short Title"]
    profile["Long Title"] = old_course_profile["Long Title"]
    if not change_profile: # scrape the old course profiles for data if old course profile exists
      profile["Course Description"] = old_course_profile["Description"]
      profile["Key Topics"] = old_course_profile["Key Topics"]
      profile["Prerequisite Knowledge"] = old_course_profile["Prerequisite Knowledge"]
      profile["Course Relevance"] = old_course_profile["Course Relevance"]
      profile["Course Goals"] = old_course_profile["Course Goals"]
      profile["Assessment Structure"] = old_course_profile["Assessment Structure"]
      profile["Learning Resources"] = old_course_profile["Learning Resources"]
      profile["Extra Time Commitments"] = old_course_profile["Extra Time Commitments"]
  else: # if we could not find the old course profile (not offered last sem), we can at least do the long title from the application
    profile["Long Title"] = app["If your course was not listed above, please fill it in below."][7:]
  return profile

def course_profile_from_app(profile, app):
  profile["Course Description"] = app["Course description (for SIO)"]
  profile["Key Topics"] = app["Key topics: What are the key subject topics that this course will cover?"]
  profile["Prerequisite Knowledge"] = app["Prerequisite Knowledge: What prior knowledge must students have in order to be successful in this course?"]
  profile["Course Relevance"] = app["Course Relevance: How is this course relevant to the targeted student populations?"]
  profile["Course Goals"] = app["Course Goals: What are the overall goals of this course that students will achieve after completing it?"]
  profile["Assessment Structure"] = app["Assessment Structure: How will students be assessed in this course: assignments, exams, final, presentation, project, etc.?"]
  profile["Learning Resources"] = app["Learning Resources: What resources will be available for students: web pages, learning applications, texts, case studies, etc.?"]
  profile["Extra Time Commitments"] = app["Extra Time Commitments: Are there extra time commitments required outside of the regularly scheduled course meeting times?"]

def get_instructor_data(application, i):
  instructor = dict()
  instructor["Instructor First Name"] = application[f"Instructor {i} First Name"]
  instructor["Instructor Last Name"] = application[f"Instructor {i} Last Name"]
  instructor["Instructor AndrewID"] = application[f"Instructor {i} Andrew ID"]
  instructor["Instructor Email"] = instructor["Instructor AndrewID"] + "@andrew.cmu.edu"
  instructor["Contract Link"] = application[f"Instructor {i} Contract"]
  instructor["Advisor Name"] = application[f"Instructor {i} Academic Advisor Name"]
  instructor["Advisor AndrewID"] = application[f"Instructor {i} Academic Advisor Andrew ID"]
  instructor["Recommending Faculty Name"] = application[f"Instructor {i} Recommending Faculty Name (not your Academic Advisor)"]
  instructor["Recommending Faculty AndrewID"] = application[f"Instructor {i} Recommending Faculty Andrew ID"]
  return instructor

def other_data(application):
  fill_data = {
      "Syllabus Link":"Upload your syllabus",
      "Room Type":"What type of room do you need?",
      "Preferred Room/Notes":"Do you have a room in particular that you would prefer?",
      "Tentative Cap":"Max enrollment for course",
      "Day":"Day of week",
      "Start Time":"Start Time (earliest is 7:00pm)",
      "End Time":"End Time",
      "Fee":"Course Fee",
      "Course Description":"Course description (for SIO)",
      "FCE Opt-In":"Would you like to receive FCEs at the end of the semester for this course?"
  }
  filled_data = dict()
  for entry in fill_data:
    filled_data[entry] = application[fill_data[entry]]
  return filled_data

get the data from each application and push it onto the new sheet

In [ ]:
def build_entries_from_application(application: dict, default_num, instr_header, profile_header, stuco_list, course_profiles):
  # information that is the same for all instructors
  course_data = other_data(application)
  course_data["Course Number"], new_default = gen_course_num(application, default_num)
  course_data = course_data | course_profile(application, course_data["Course Number"])

  # copy data for each instructor
  instr_count = int(application["Number of Instructors"])+1
  for instr in range(1, instr_count):
    entry = course_data | get_instructor_data(application, instr)
    enter_instr(entry, stuco_list, instr_header)

  # enter data to course profile sheet
  profile = course_data
  profile["Class Number"] = course_data["Course Number"]
  profile["Description"] = course_data["Course Description"]
  enter_course_profile(profile, course_profiles, profile_header)
  return new_default

def enter_instr(entry: dict, sheet, header):
  row = [""] * len(header)
  for i, col in enumerate(header):
    if col in entry: row[i] = entry[col]
  sheet.append_row(row)

def enter_course_profile(entry: dict, sheet, header):
  row = [""] * len(header)
  for i, col in enumerate(header):
    if col in entry: row[i] = entry[col]
  sheet.append_row(row)

function to build the stuco list from the form response sheet

In [ ]:
# builds the whole list given locations of relevant stuff
def build_list(form_response, stuco_list):
  instr_sheet_columns = 30
  instr_header = gc.open_by_url(previous_stuco_list_url).get_worksheet(0).get("1:1")[0][0:instr_sheet_columns] # retrieve header from previous stuco list, but cut off audit stuff
  instructor_list = stuco_list.get_worksheet(0)
  instructor_list.insert_row(instr_header, 1)
  instructor_list.delete_rows(2, 500)

  profile_header = gc.open_by_url(previous_stuco_list_url).get_worksheet(1).get("1:1")[0]
  course_profiles = stuco_list.get_worksheet(1)
  course_profiles.insert_row(profile_header, 1)
  course_profiles.delete_rows(2, 200)

  applications = gc.open_by_url(form_response).worksheet("Form Responses 1").get_all_records()

  default = 901
  for application in applications:
    default = build_entries_from_application(application, default, instr_header, profile_header, instructor_list, course_profiles)

  instructor_list.batch_format([{"range":"2:300", "format":{"wrapStrategy":"CLIP"}}])
  instructor_list.delete_columns(instr_sheet_columns, 50)

actually run everything

In [ ]:
if new_stuco_list_url == None:
  new_stuco_list = gc.create(f"{semester} StuCo List", folder_id)
  new_stuco_list.add_worksheet(f"{semester} Course Profile", 1, 11)
else:
  new_stuco_list = gc.open_by_url(new_stuco_list_url)

new_stuco_list.get_worksheet(0).batch_clear(["1:1000"])
new_stuco_list.get_worksheet(1).batch_clear(["1:1000"])

build_list(application_form, new_stuco_list)

501
502
503
504
505
506
507
